In [ ]:
import os
import requests
import json
import re
import pandas as pd
import pandasql as psql
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import csv


# 手動で環境変数を設定
os.environ['GEMINI_API_KEY'] = "Your API Key"
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    raise ValueError("GEMINI_API_KEY が設定されていません。")

# API エンドポイント
url = f"https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash-latest:generateContent?key={api_key}"
# データの読み込み
course_info = pd.read_csv('course_info.csv')

# モデルの読み込み
model = SentenceTransformer('sonoisa/sentence-bert-base-ja-mean-tokens-v2')
# テキストの統合　
course_info = course_info.fillna('')  # 欠損値を空文字列に置き換え
course_info['text'] = (
    course_info['科目名'].astype(str) + ' ' + course_info['単位'].astype(str) + ' ' + course_info['研究会テーマ'].astype(str) + ' ' +
    course_info['曜日・時限'].astype(str) + ' ' + course_info['授業で英語サポート'].astype(str) + ' ' + course_info['履修条件'].astype(str) + ' ' +
    course_info['その他推奨知識'].astype(str) + ' ' + course_info['履修制限'].astype(str) + ' ' + course_info['課題提出タイプ'].astype(str) + ' ' +
    course_info['学生が利用する予定機材/ソフト等'].astype(str) + ' ' + course_info['履修上の注意・留意事項'].astype(str) + ' ' +
    course_info['参考文献'].astype(str) + ' ' + course_info['講義概要'].astype(str) + ' ' + course_info['主題と目標'].astype(str) + ' ' +
    course_info['第1回'].astype(str) + ' ' + course_info['第2回'].astype(str) + ' ' + course_info['第3回'].astype(str) + ' ' +
    course_info['第4回'].astype(str) + ' ' + course_info['第5回'].astype(str) + ' ' + course_info['第6回'].astype(str) + ' ' +
    course_info['第7回'].astype(str) + ' ' + course_info['第8回'].astype(str) + ' ' + course_info['第9回'].astype(str) + ' ' +
    course_info['第10回'].astype(str) + ' ' + course_info['第11回'].astype(str) + ' ' + course_info['第12回'].astype(str) + ' ' +
    course_info['第13回'].astype(str) + ' ' + course_info['第14回'].astype(str) + ' ' + course_info['第15回'].astype(str) + ' ' +
    course_info['その他'].astype(str)
)
corpus = course_info['text'].tolist()
corpus_embeddings = model.encode(corpus, batch_size=64, show_progress_bar=True)


corpus_embeddings = np.array(corpus_embeddings).astype('float32')
dimension = corpus_embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)  # 内積ベースの類似度（コサイン類似度に対応）
index.add(corpus_embeddings)
corpus_embeddings_normalized = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)
index.add(corpus_embeddings_normalized)

# プロンプトデータを作成
def generate_with_gemini(prompt):
    headers = {
        'Content-Type': 'application/json',
    }

    data = {
        "contents": [
            {
                "parts": [
                    {"text": prompt}
                ]
            }
        ]
    }

    # API リクエストの送信
    response = requests.post(url, headers=headers, json=data)

    # 応答の処理
    if response.status_code == 200:
        try:
            return response.json()["candidates"][0]["content"]["parts"][0]["text"]
        except (KeyError, IndexError):
            return "Response structure is not as expected."
    else:
        return f"Error: {response.status_code}, {response.text}"
    
def generate_prompt_1(user_input_prompt):
    prompt = f'''あなたは、大学のシラバスRIGシステムに統合されたAIです。以下の処理フローに従って、回答を生成してください。なお、この応答に対して生成したSQLクエリやコサイン類似度検索の結果はユーザーの質問と統合して再び2段階的にgemini_apiへ応答させることを念頭に置いてください。

1. ユーザーの質問を解析し、SQLクエリを生成・実行する必要があるかを判断します。SQLクエリを生成・実行する必要がある場合は、変数:whether_sql に1を、必要がない場合は0を代入します。

2. 変数:whether_sqlの値に応じて、以下の処理を行います。変数:whether_sqlが1の場合、ユーザーの質問を解析し、データベースのスキーマ情報を参照してSQLクエリを生成し、変数:sql_queryに代入します。変数:whether_sqlが0の場合、変数:sql_queryには0を代入します。ORDER BY RANDOM()を使用して、ランダムな結果を返すことを忘れないでください。なお、SQLの出力に幅を持たせるため、なるべく=ではなくLIKEを使用することをお勧めします。また、質問の文脈に応じて~に関連するなどの広い範囲を聞かれる場合は文章を持つプロパティからのLIKE検索も複数行うことをお勧めします。

3. 次に、ユーザーの質問の内容からコサイン類似度検索を行う必要があるか判断します。コサイン類似度検索を実行する必要がある場合は、変数:whether_cosine_search に1を代入します。必要がない場合は0を代入します。

4. 変数:whether_cosine_search の値に応じて以下の処理を行います。変数:whether_cosine_search が1の場合、以下のフローで`cosine_search_text`を生成します：
    - ユーザーの質問から具体的なキーワードを抽出します。
    - 抽出されたキーワードが授業に関連するものであれば、それを簡潔にまとめたテキストを生成します。
    - 例：ユーザーの質問が「気象にまつわる授業を教えてください」というものであれば、関連するキーワード「気象、気候、環境、天候」などを抽出し、`cosine_search_text`に「気象、気候、環境、天候の授業」と代入します。
    - 変数:whether_cosine_search が0の場合、変数:cosine_search_textには0を代入します。

5. 結果をJSON形式で出力します。出力形式は次の通りです。ここに示す以外の情報は出力しないでください。：

    - whether_sql: whether_sqlの値
    - sql_query: sql_queryの値
    - whether_cosine_search: whether_cosine_searchの値
    - cosine_search_text: cosine_search_textの値

ユーザの質問: {user_input_prompt}、データベースのスキーマ情報は以下の通りです:
[スキーマ情報]course_info.csv
    schema_details = （
        '学部・研究科': '総合政策・環境情報学部',
        '登録番号': '3~5桁からなる科目ごとの登録番号',
        '科目ソート': '科目の種類ID',
        '科目名': '授業の名前',
        '分野': '授業の分野（例：気象学、プログラミング）',
        '単位': '授業の単位数、1単位、2単位、4単位',
        '開講年度・学期': '授業が開講される年度と学期',
        'K-Number': '授業のK-NumberID',
        '研究会テーマ': '科目名に研究会という文字列が含まれている場合のみ記載',
        '曜日・時限': '授業の曜日と時限（例：月 4限    , 月 5限）',
        ’実施形態’: '授業の実施形態（例：対面、オンライン）',
        '授業で使う言語': '授業で使う言語（例：日本語、英語）',
        '授業で英語サポート': '授業で英語サポートあり、授業で英語サポートなし',
        '履修条件': '授業の履修条件、文章で記載',
        'その他推奨知識': '授業を受講する際に推奨される知識、文章で記載',
        '開講場所': '授業を開講する担当者の所属場所',
        '履修制限': '授業の履修制限、文章で記載',
        '受け入れ予定人数': '授業の受け入れ予定人数float型',
        '課題提出タイプ': '課題提出のタイプ',
        '学生が利用する予定機材/ソフト等': '学生が利用する予定の機材やソフトウェア',
        '履修上の注意・留意事項': '授業を受講する際に留意すべき事項,文章で記載',
        '参考文献': '授業で参考にすべき文献、文章で記載',
        '連絡先メールアドレス': '授業に関する問い合わせ先のメールアドレス',
        'GIGAサティフィケート対象': 'GIGAサティフィケート対象かどうか',
        '講義概要': '授業の説明,文章で記載',
        '主題と目標': '授業の目標やテーマ、文章で記載',
        '第1回': '授業の第1回の内容、文章で記載',
        '第2回': '授業の第2回の内容、文章で記載',
        ///略
        '第15回': '授業の第15回の内容、文章で記載',
        'その他': 'その他の情報、文章で記載'
    ）
   '''
    return prompt

# クエリの取得
def get_query(response):
    cleaned_response = response[:3] + response[7:]
    cleaned_response = cleaned_response[3:-3] 
    parsed_data = json.loads(cleaned_response)
    whether_sql = parsed_data.get('whether_sql')
    sql_query = parsed_data.get('sql_query')
    whether_cosine_search = parsed_data.get('whether_cosine_search')
    cosine_search_text = parsed_data.get('cosine_search_text')
    return whether_sql, sql_query, whether_cosine_search, cosine_search_text


def sql_query_execution(whether_sql, sql_query, course_info):
    # SQLクエリの実行
    if whether_sql == 1:
        # SQLクエリの実行
        sql_result = psql.sqldf(sql_query, locals())
        if sql_result.empty:
            sql_result = "SQLクエリによる結果はありません。"
    else:
        sql_result = "SQLクエリによる結果はありません。"
    
    return sql_result

def cosine_search(whether_cosine_search, cosine_search_text):
    if whether_cosine_search == 1:
        # コサイン類似度検索
        query_embedding = model.encode([cosine_search_text], convert_to_tensor=True)
        # PyTorchのnorm関数を使用
        query_embedding = query_embedding / query_embedding.norm(p=2, dim=1, keepdim=True)
        # テンソルをCPU上のNumPy配列に変換
        query_embedding = query_embedding.cpu().numpy().astype('float32')
        # 検索の実行
        D, I = index.search(query_embedding, k=5)
        cosine_search_result = course_info.iloc[I[0], :]
        similar_courses = course_info.iloc[I[0]]
        threshold = 0.7
        valid_indices = D[0] >= threshold
        filtered_courses = similar_courses[valid_indices]
        cosine_search_result = filtered_courses
        if cosine_search_result.empty:
            cosine_search_result = "コサイン類似度検索による結果はありません。"
    else:
        cosine_search_result = "コサイン類似度検索による結果はありません。"

    return cosine_search_result

def generate_prompt_2(user_input_prompt, result_sql, result_cosine_search):
    prompt = f'''以下の結果を元に、ユーザーの質問に対する回答を生成してください。
    ユーザーの質問：{user_input_prompt}、
    SQLクエリによる結果:{result_sql}、
    コサイン類似度検索による結果：{result_cosine_search}
    '''
    return prompt


In [ ]:
def main():
    user_input_prompt = '研究会を除いた、気象に関連する授業はいくつある？'
    prompt_1 = generate_prompt_1(user_input_prompt)
    response_1 = generate_with_gemini(prompt_1)
    whether_sql, sql_query, whether_cosine_search, cosine_search_text = get_query(response_1)
    result_sql = sql_query_execution(whether_sql, sql_query, course_info)
    result_cosine_search = cosine_search(whether_cosine_search, cosine_search_text)
    prompt_2 = generate_prompt_2(user_input_prompt, result_sql, result_cosine_search)
    response_2 = generate_with_gemini(prompt_2)
    print(response_2)

if __name__ == '__main__':
    main()